In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor

In [ ]:
df = pd.read_csv("./data/_output.csv")
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(df[["Close"]])

In [ ]:
train_size = int(len(data) * 0.75)
train_set = data[:train_size]
test_set = data[train_size:]

look_back = 150
n_steps = 5
x_train, y_train = [], []
for i in range(len(train_set) - look_back - n_steps):
    x_train.append(train_set[i : (i + look_back), 0])
    y_train.append(train_set[i + look_back : i + look_back + n_steps, 0])
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test, y_test = [], []
for i in range(len(test_set) - look_back - n_steps):
    x_test.append(test_set[i : (i + look_back), 0])
    y_test.append(test_set[i + look_back : i + look_back + n_steps, 0])
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
def create_model(units, dropout, learning_rate):
    model = Sequential()
    model.add(Input(shape=(look_back, 1)))
    model.add(LSTM(units, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(units))
    model.add(Dropout(dropout))
    # model.add(Dense(n_steps * 2))
    model.add(Dense(n_steps))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mean_squared_error")
    return model


# model = KerasRegressor(model=create_model, units=32, dropout=0.05, learning_rate=0.1, verbose=0)
# param_grid = {
#     "units": [32, 64, 128],
#     "dropout": [0.05],
#     "learning_rate": [1e-3],
#     "batch_size": [16, 32, 64],
#     "epochs": [500]
# }
# random_search = RandomizedSearchCV(model, param_grid, n_iter=8, cv=3, n_jobs=-1, verbose=0)
# random_search.fit(x_train, y_train)
# print("random_search.best_params_: ", random_search.best_params_)
# print("random_search.best_score_: ", random_search.best_score_)

In [ ]:
# best_params = random_search.best_params_
best_params = {"units": 64, "learning_rate": 0.001, "epochs": 500, "dropout": 0.05, "batch_size": 32}

In [ ]:
best_model = create_model(units=best_params["units"], dropout=best_params["dropout"], learning_rate=best_params["learning_rate"])
best_model.fit(x_train, y_train, batch_size=best_params["batch_size"], epochs=best_params["epochs"])

In [ ]:
y_pred = scaler.inverse_transform(best_model.predict(x_test))
y_test = scaler.inverse_transform(y_test)
print("MSE score: ", mean_squared_error(y_test, y_pred))
print("RMSE score: ", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE score: ", mean_absolute_error(y_test, y_pred))

In [ ]:
best_model.save("./HSI_LSTM_Analysis_Multiple_Output.keras")